#Problem 1 - Looking Back at Scratch

Initialization of the weights and Bias

*   Epoch loop was needed
*   Number of nodes were determined and activation for each layer was performed
*   Training data was the mini-batch size
*   Assigned the linearly combined value to the activation function and passed the results to the next later (Forward Propagation)
*Computed the Loss Function
*Updated the weights and Biases (Error Back Propagation)
*Predicted with the validation data using the learned parameters











*Problem 2 - Consider the correspondence between scratch and TensorFlow

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.test.gpu_device_name() 

''

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [5]:
class GetMiniBatch:
    """
    Iterator to get a mini-batch

    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [7]:
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    #Declaration of weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    # tf.add and + are equivalent
    
    return layer_output

In [ ]:
"""
Binar classification of Iris dataset using neural network implemented in TensorFlow
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#Load dataset
dataset_path = "./Iris/Iris.csv"
df = pd.read_csv(dataset_path)

#Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

#Convert labels to numbers
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]

#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

#Further split into train and Validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#Read network structure                              
logits = example_net(X)

# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

#Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initialize the variable
init = tf.global_variables_initializer()


#Run computational graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))


#Problem 3 - Create a model of Iris using all three types of objective variables

In [ ]:
#Load dataset
dataset_path = "./Iris/Iris.csv"
df_iris = pd.read_csv(dataset_path)

#Condition extraction from data frame
df_iris = df_iris[(df_iris["Species"] == "Iris-setosa") 
                  |(df_iris["Species"] == "Iris-versicolor")
                  |(df_iris["Species"] == "Iris-virginica")
                  ]

y = df_iris["Species"]
X = df_iris.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

#Convert labels to numbers
y[y =='Iris-setosa'] = 0
y[y =='Iris-versicolor'] = 1
y[y =='Iris-virginica'] = 2

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:, np.newaxis])


#Split into train and test
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.2, random_state=0)

#Further split into train and Validation
X_train, X_val, y_train, y_val = \
train_test_split(X_train, y_train, test_size=0.2, random_state=0)


In [ ]:
print('y.shape:', y.shape)
print('X.shape:', X.shape)

In [ ]:
#Plot the Train Data
import seaborn as sns
sns.pairplot(df_iris,hue='Species');

In [ ]:
# Configure another set of Hyperparameter
learning_rate = 0.001
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#Read network structure                              
logits = example_net(X)

# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                         labels=Y, logits=logits)
                        )

#Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Estimated result
correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(tf.sparse_softmax(logits),1))

#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initialize the variable
init = tf.global_variables_initializer()


#Run computational graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run(
                [loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y}
            )
            total_loss += loss
            total_acc += acc

        total_loss /= n_samples
        total_acc /= n_samples

        val_loss, val_acc = sess.run(
            [loss_op, accuracy], feed_dict={X: X_val, Y: y_val}
        )
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}"
              .format(epoch, loss, val_loss, acc, val_acc))
    
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))



#Problem 4 - Create a model of House Prices
Create a model using House Prices, a data set for regression problems

In [ ]:
#Loading the dataset
dataset_path = "house_prices_advanced_regression_technique/train.csv"
df_house = pd.read_csv(dataset_path)

#Extracting Conditions from the data frame
y = df_house[['SalePrice']]
X = df_house[['GrLivArea', 'YearBuilt']]
y = np.array(np.log1p(y))
X = np.array(np.log1p(X))

#Split into train and test
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.2, random_state=0)

#Further split into train and Validation
X_train, X_val, y_train, y_val = \
train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
print('y.shape:', y.shape)
print('X.shape:', X.shape)

In [ ]:
#Plotting the Train Data
import matplotlib as plt

plt.rcParams["font.size"] = 20
fig,ax = plt.subplots(1,2,sharey=True,figsize=(16, 5))

ax[0].scatter(X_train[:,0],y_train)
ax[1].scatter(X_train[:,1],y_train);

In [ ]:
def regression_net(x):

    # Declaring the weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }

    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    #tf.add and + are equivalent
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']

    return layer_output

In [ ]:
# Configure another set of Hyperparameter
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder(tf.float, [None, n_input])
Y = tf.placeholder(tf.float, [None, n_classes])

# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#Read network structure                              
logits = example_net(X)

# Objective function
loss_op = tf.reduce_mean(tf.square(logits=logits - Y))

#Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Index Value Calculation
mean_square_error = tf.reduce_mean(tf.square(logits=logits - Y))

#Initialize the variable
init = tf.global_variables_initializer()


#Run computational graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_mse = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})

            loss, mse = sess.run([loss_op, mean_square_error], 
                                 feed_dict={X: mini_batch_x, Y: mini_batch_y}
            )
            total_loss += loss
            total_mse += mse

        total_loss /= n_samples
        total_mse /= n_samples

        val_loss, val_mse = sess.run([loss_op, mean_square_error], 
                                     feed_dict={X: X_val, Y: y_val}
        )
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, mse : {:.3f}, val_mse : {:.3f}"
              .format(epoch, loss, val_loss, mse, val_mse))
        
    test_mse = sess.run(mean_square_error, feed_dict={X: X_test, Y: y_test})
    print("test_mse : {:.3f}".format(test_mse))


#Problem 5 - Create a model of MNIST
Create a model to classify the MNIST used in neural network scratches.

*It is similar to the previous Iris in that it is classified into 3 or more classes. The difference is that the input is an image.

*Aim to reproduce the model mounted by scratch.

In [ ]:
# Download the MNIST Dataset
from keras.datasets import mnist
(x,y), (X_test, y_test) = mnist.load_data()


In [ ]:
#Checking the Data imputed
print(X.shape)
print(X.shape)
print(X[0].dtype)

In [ ]:
#Smooting
X = X.reshape(-1, 784)
X_test = X_test.reshape(-1,784)
print(X.shape) 
print(X_test.shape)

In [ ]:
 #Type Conversion, Normalization
X = X.astype(np.float)
X_test = X_test.astype(np.float)
X /= 255
X_test /= 255
print(x.max())
print(x.min())

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])
print(y.shape)
print(y_one_hot.shape)
print(y_one_hot.dtype)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y_one_hot, test_size=0.2)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

In [ ]:
def lenet(x):
    """
    CNN
    """
    #Declaration of weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([5,5,1,6])),
        'w2': tf.Variable(tf.random_normal([5,5,6,16])),
        'w3': tf.Variable(tf.random_normal([7*7*16, 120])),
        'w4': tf.Variable(tf.random_normal([120,84])),
        'w5': tf.Variable(tf.random_normal([84, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([6])),
        'b2': tf.Variable(tf.random_normal([16])),
        'b3': tf.Variable(tf.random_normal([120])),
        'b4': tf.Variable(tf.random_normal([84])),
        'b5': tf.Variable(tf.random_normal([84, n_classes]))
    }

    X = tf.reshape(X, [-1, 28, 28, 1])
    conve_1 = tf.add(tf.nn.conv2d(X, weights['w1'],strides=[1,1,1,1],
                                 padding='SAME'),
                    biases['b1'])
    
    conve_1 = tf.nn.relu(conve_1)

    poolin_1 = tf.nn.pool(conve_1, window_shape=[2,2],strides=[2,2],
                          pooling_type='MAX', padding='VALID')
    
    conve_2 = tf.add(tf.nn.conv2d(poolin_1, weights['w2'],strides=[1,1,1,1],
                                  padding='SAME'),
                     biases['b2'])
    
    conve_2 = tf.nn.relu(conve_2)

    poolin_2 = tf.nn.pool(conve_2, window_shape=[2,2],strides=[2,2],
                          pooling_type='MAX', padding='VALID')
    
    X_reshape = tf.reshape(poolin_2, [-1,7*7*16])
    layer_1 = tf.add(tf.matmul(X_reshape, weights['w3']), biases['b3'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(X_reshape, weights['w4']), biases['b4'])
    layer_2 = tf.nn.relu(layer_2)
    # tf.add and + are equivalent
    layer_output = tf.matmul(layer_2, weights['w5']) + biases['b5']
    
    return layer_output

In [ ]:
# Configure another set of Hyperparameter
learning_rate = 0.01
batch_size = 200
num_epochs = 30

n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#Read network structure                              
logits = lenet(X)

# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                          labels=Y, logits=logits - Y)
                          )

#Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Estimated result
correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(tf.nn_softmax(logits),1))

#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initialize the variable
init = tf.global_variables_initializer()


#Run computational graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})

            loss, acc = sess.run(
                [loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc

        total_loss /= n_samples
        total_acc /= n_samples

        val_loss, val_acc = sess.run(
            [loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}"
              .format(epoch, loss, val_loss, acc, val_acc))
        
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
